In [1]:
import sys; sys.path.append('..')
from os import path
from random import choice
from graph import Graph, ReaderORLibrary
from graph.steiner import (prunning_mst, shortest_path,
                            shortest_path_origin_prim,
                            prunning_kruskal_mst,
                            shortest_path_with_origin)

from graph.util import is_steiner_tree, has_cycle
from graph.algorithms import kruskal, prim

In [2]:
from matplotlib import pyplot as plt
from draw import draw_radial, draw_common

In [3]:
from operator import attrgetter

In [4]:
from pxcrossover import compose, connected

As heurísticas para o STPG dão uma medida de aproximação sobre a qualidade da solução próxima que encontram. 
Esta medida pode ser de alguma forma utilizada para verificar quando uma heurísticas pode ser utilizada para determinada instância de forma que o operador de cruzamento seja eficiênte?

In [5]:
file = path.join('..', 'datasets','ORLibrary', 'steinb18.txt')

path.exists(file)

True

In [6]:
reader = ReaderORLibrary()

stpg = reader.parser(file)

In [7]:
stpg.graph

In [8]:
# print(stpg.terminals)

In [9]:
help(prunning_mst)

Help on function prunning_mst in module graph.steiner:

prunning_mst(graph, start, terminals)
    Parameters:
        graph : Graph
            Base graph to compute the Steiner tree
        start : Node
            Where is to start Prim's algorithm
        terminals : Set
            Terminals Nodes from the STPG instance
    
    Return:
        prunning : A Steiner Tree
        total : Numeric - total cost of the tree
    
    Notes:
        Determina a MST do grafo por meio do algoritmo de Prim.
        Realiza a poda considerando os nós terminais como os nós folhas até a raiz <start>
        Se <start> for um vértice não terminal, o laço <while> verifica se <start> é um vértice folha da árvore resultande.
        Em caso afirmativo realiza uma poda iterativa a partir desse vértice para garantir que a árvore resultante seja
        uma árvore de Steiner.
        Resulta sempre na mesma árvore para qualquer vértice <start> considerado.



In [10]:
help(is_steiner_tree)

Help on function is_steiner_tree in module graph.util:

is_steiner_tree(subgraph: graph.graph.Graph, STPG: graph.reader.SteinerTreeProblem)
    Check if the graph passed is a Steiner Tree.
    
    Parameters:
        subgraph : Graph
            represent the partial solution to be tested
        STPG : SteinerTreeProblem
            the problem instance itself
    
    Returns:
        bool: True or False
        dict : status' test
    
    Note: A Steiner Tree might not be a Minimal Steiner Tree.



In [11]:
%%time

tt = choice(list(stpg.terminals))

aa, a_cost = prunning_mst(stpg.graph, tt, stpg.terminals)

a_cost

Wall time: 2 ms


227

In [12]:
is_steiner_tree(aa,stpg)

(True,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': True})

In [13]:
%%time 

tt = choice(list(stpg.terminals))
terminals = set(stpg.terminals)

cc = prunning_kruskal_mst(stpg.graph, terminals)

is_steiner_tree(cc, stpg)

Wall time: 3 ms


(True,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': True})

In [14]:
non_terminal = [ v for v in range(1, stpg.nro_nodes + 1) if v not in stpg.terminals]
# print(non_terminal)

In [15]:
v = choice(non_terminal)

print("vertice ", v)

bb, b_cost = prunning_mst(stpg.graph, v, stpg.terminals)

b_cost

vertice  30


225

In [16]:
is_steiner_tree(bb, stpg)

(True,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': True})

In [17]:
v in bb

True

In [18]:
help(compose)

Help on function compose in module pxcrossover:

compose(red: graph.graph.Graph, blue: graph.graph.Graph)
    Parameters:
    ----------
        red, blue : Graph
    
    Return:
    -------
        g_union, g_common, g_star : Graph



In [19]:
g_union, g_common, g_star = compose(aa, bb)

In [20]:
help(connected)

Help on function connected in module pxcrossover:

connected(g_union: graph.graph.Graph, red: graph.graph.Graph, blue: graph.graph.Graph, start: 'node')



In [21]:
%%time

first, second, previous = connected(g_union, aa, bb, tt)

Wall time: 998 µs


In [22]:
len(first)

2

In [23]:
len(second)

2

In [24]:
# plt.figure(figsize=(15,15))

# draw_common(g_union, stpg.terminals, aa, bb)

In [25]:
# os vértices diferentes possuem os mesmos pais
def test_2(right, left, commonset):
    
    diff_right = right.portal - left.portal
    diff_left  = left.portal - right.portal
    
    portal_right = set(map(lambda x : commonset.find(x), diff_right))
    portal_left  = set(map(lambda x : commonset.find(x), diff_left))
    
    return portal_right == portal_left

In [26]:
first

In [27]:
second

In [28]:
matches = dict()

for component in first:
    ss = frozenset(previous.find(v) for v in component.portal)
    matches[ss] = [component]

In [29]:
non_second = list()

for component in second:
    ss = frozenset(previous.find(v) for v in component.portal)
    if ss in matches:
        matches[ss].append(component)
    else:
        non_second.append(component)

In [30]:
# component.edges

In [31]:
len(matches)

2

In [32]:
matches

{frozenset({3, 57}): [<pxcrossover.Component at 0x2165ea4cc40>,
 frozenset({3, 82}): [<pxcrossover.Component at 0x2165ea4cca0>]}

In [33]:
get_cost = attrgetter('cost')

success = 0
fail = 0

non_first = list()
how_many = list()

minimum_cost = list()

for key, components in matches.items():
    if len(components) >= 2:
        how_many.append(len(components))
        success += 1 
        minimum_cost.append(min(components, key=get_cost))
    else:
        print(len(components), end='\t')
        non_first.append(components.pop())
        fail += 1

1	

In [34]:
if len(minimum_cost) < 7:
    for c in minimum_cost: 
        print(c, c.cost, c.portal, stpg.graph.has_edge(*c.portal))

<pxcrossover.Component object at 0x000002165EA4CC40> 1 {18, 19} True


In [35]:
len(minimum_cost)

1

In [36]:
get_cost(component)

5

In [37]:
how_many

[2]

In [38]:
success

1

In [39]:
fail

1

In [40]:
len(non_second)

1

In [41]:
len(non_first)

1

In [42]:
total_first = 0
for c in non_first:
#     print(c, c.cost, c.portal, stpg.graph.has_edge(*c.portal))
    total_first += c.cost

In [43]:
total_second = 0
for c in non_second: 
#     print(c, c.cost, c.portal, stpg.graph.has_edge(*c.portal))
    total_second += c.cost

In [44]:
print(total_first)
print(total_second)

9
5


In [45]:
if total_second < total_first:
    selected = non_second
else:
    selected = non_first

In [46]:
for component in minimum_cost:
    print(component.cost)
        
    print("------")

1
------


In [47]:
minimum_cost

In [48]:
for component in minimum_cost:
    for v, u in component.edges.items():
        if u is None:
            continue
        w = stpg.graph.W(v,u)
        g_common.add_edge(v,u,weight=w)

In [49]:
is_steiner_tree(g_common, stpg)

(False,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': False})

In [50]:
len(selected)

1

In [51]:
for component in selected:
    for v, u in component.edges.items():
        if u is None:
            continue
        w = stpg.graph.W(v,u)
        g_common.add_edge(v,u,weight=w)

In [52]:
is_steiner_tree(g_common, stpg)

(True,
 {'has_cycle': False,
  'all_terminals_in': True,
  'all_leaves_are_terminals': True,
  'all_edges_are_reliable': True,
  'graph_is_connected': True})

In [53]:
# g_common.edges

In [54]:
from graph.util import gg_total_weight

In [55]:
gg_total_weight(g_common)

225

In [56]:
gg_total_weight(aa)

227

In [57]:
gg_total_weight(bb)

225